<a href="https://colab.research.google.com/github/ritwiks9635/My_priject_file_5/blob/main/dog_breed_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORT MODULE

In [ ]:
import cv2
import numpy as np 
import pandas as pd
import tensorflow as tf 
from tensorflow import keras 
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from keras.preprocessing import image
import warnings 
warnings.filterwarnings("ignore")

Import dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
labels = pd.read_csv("/content/drive/MyDrive/labels.csv")
train = "/content/drive/MyDrive/train"
test = "/content/drive/MyDrive/test"

In [ ]:
labels.head(4)

Adding ‘.jpg’ extension to each id

In [ ]:
def to_jpg(id):
  return id + ".jpg"

labels["id"] = labels["id"].apply(to_jpg)

In [ ]:
labels.head()

Augmenting data

In [ ]:
data_gen = ImageDataGenerator(
  rescale = 1./255, 
  horizontal_flip = True,
  validation_split=0.2)

In [ ]:
train_gen = data_gen.flow_from_dataframe(
  labels,
  directory = train,
  x_col = "id",
  y_col = "breed",
  subset = "training",
  color_mode = "rgb",
  target_size = (331, 331),
  class_mode = "categorical",
  batch_size = 32,
  shuffle = True,
  seed = 42)

val_gen = data_gen.flow_from_dataframe(
  labels,
  directory = train,
  x_col = "id",
  y_col = "breed",
  subset = "validation",
  color_mode = "rgb",
  target_size = (331, 331),
  class_mode = "categorical",
  batch_size = 32,
  shuffle = True,
  seed = 42)

In [ ]:
x, y = next(train_gen)
x.shape

(32, 331, 331, 3)

Plotting images from the train dataset

In [ ]:
a = train_gen.class_indices
class_name = list(a.keys())
def plot_img(image, labels):
  plt.figure(figsize=(15,10))
  for i in range(25):
    plt.subplot(5,5,i+1)
    plt.imshow(image[i])
    plt.title(class_name[np.argmax(labels[i])])
    plt.axis("off")

In [ ]:
plot_img(x,y)

Building our Model

In [ ]:
# load the InceptionResNetV2 architecture with imagenet weights as base
base_model = applications.InceptionResNetV2(
  include_top=False,
  weights = "imagenet",
  input_shape = (331,331,3))
base_model.trainable=False

219055592/219055592 [==============================] - 11s 0us/step


In [ ]:
model = Sequential([
  base_model,
  tf.keras.layers.BatchNormalization(renorm = True),
  GlobalAveragePooling2D(),
  Dense(512, activation = "relu"),
  Dense(256, activation = "relu"),
  Dropout(0.5),
  Dense(128, activation = "relu"),
  Dense(120, activation = "softmax")])

Compile the model

In [ ]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

Displaying a summary report of the model

In [ ]:
model.summary()

Defining callbacks to preserve the best results

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(patience = 10, min_delta = 0.001, restore_best_weights = True)

Training the model

In [ ]:

batch_size=32
STEP_SIZE_TRAIN = train_gen.n//train_gen.batch_size
STEP_SIZE_VALID = val_gen.n//val_gen.batch_size
 
# fit model
history = model.fit(train_gen,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=val_gen,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=25,
                    callbacks=[early_stop])

Visualizing the model’s performance

In [ ]:

# store results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
 
# plot results
# accuracy
plt.figure(figsize=(10, 16))
plt.rcParams['figure.figsize'] = [16, 9]
plt.rcParams['font.size'] = 14
plt.rcParams['axes.grid'] = True
plt.rcParams['figure.facecolor'] = 'white'
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title(f'\nTraining and Validation Accuracy. \nTrain Accuracy:
          {str(acc[-1])}\nValidation Accuracy: {str(val_acc[-1])}')

In [ ]:

# loss
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title(f'Training and Validation Loss. \nTrain Loss:
          {str(loss[-1])}\nValidation Loss: {str(val_loss[-1])}')
plt.xlabel('epoch')
plt.tight_layout(pad=3.0)
plt.show()

Evaluating the accuracy of the model

In [ ]:

accuracy_score = model.evaluate(val_gen)
print(accuracy_score)
print("Accuracy: {:.4f}%".format(accuracy_score[1] * 100))
 
print("Loss: ",accuracy_score[0])